### 데이터셋 catanddog로 resnet

In [1]:
import torchvision.transforms.v2 as v2
import torchvision
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchsummary import summary


In [2]:
import os
from pathlib import Path

train_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.CenterCrop(size=(224, 224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset= ImageFolder('./080289/chap06/data/catanddog/train', transform= train_transform)
test_dataset= ImageFolder('./080289/chap06/data/catanddog/test', transform=test_transform)


train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

C:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
from torchvision.models import resnet50, ResNet50_Weights
model_v2 =resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [4]:
for p in model_v2.parameters():
    p.requires_grad=False

In [5]:
for p in model_v2.fc.parameters():
    p.requires_grad=True

In [6]:
model_v2.fc =nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(512, 32),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(32, 2),
    )
model_v2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
summary(model_v2)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Bottleneck: 2-1                   --
|    |    └─Conv2d: 3-1                  (4,096)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─Conv2d: 3-3                  (36,864)
|    |    └─BatchNorm2d: 3-4             (128)
|    |    └─Conv2d: 3-5                  (16,384)
|    |    └─BatchNorm2d: 3-6             (512)
|    |    └─ReLU: 3-7                    --
|    |    └─Sequential: 3-8              (16,896)
|    └─Bottleneck: 2-2                   --
|    |    └─Conv2d: 3-9                  (16,384)
|    |    └─BatchNorm2d: 3-10            (128)
|    |    └─Conv2d: 3-11                 (36,864)
|    |    └─BatchNorm2d: 3-12            (128)
|    |    └─Conv2d: 3-13                 (16,384)
|    | 

In [8]:
torch.__version__, torch.cuda.is_available()

('2.4.0', False)

In [15]:
def train(model, data_loader, loss_fn, optimizer, EPOCHS=1):
    loss_history = []
    acc_history = []
    epoch_loss = 0.
    num_corrects = 0
    best_acc= 0.
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for n in range(EPOCHS):
        step_loss = 0
        num_corrects = 0
        for idx, (X_train, y_label) in enumerate(data_loader):
            inputs = X_train.to(device)
            labels = y_label.to(device)
            model = model.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            step_loss += loss.item()
            #print(torch.argmax(outputs, dim=1))
            #print(y_label)
            train_acc = torch.sum(torch.argmax(outputs, dim=1) == labels)
            num_corrects += train_acc
            #if idx % 5 == 0:
            #epoch_loss = step_loss / len(X_train) # loss per batch
            #print('  batch {} loss: {}, acc: {}'.format(idx + 1, step_loss, train_acc/len(X_train)))
            #step_loss = 0
        epoch_loss = step_loss / len(data_loader)
        accuracy = num_corrects / len(data_loader.dataset)
        print('EPOCH {}, loss: {}, acc: {}'.format(n + 1, epoch_loss, accuracy))

        if accuracy > best_acc:
            best_acc = accuracy

        acc_history.append(accuracy.item())
        loss_history.append(epoch_loss)        
        torch.save(model.state_dict(), os.path.join('./080289/chap06/data/catanddog/', '{0:0=2d}.pth'.format(n)))
   

In [16]:
params_to_update = []
for name,param in model_v2.named_parameters():
    if param.requires_grad == True:
       params_to_update.append(param)
     

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params_to_update)


In [18]:
train(model_v2, train_data_loader, loss_fn, optimizer, 5)

EPOCH 1, loss: 0.12168735112337505, acc: 0.9566037654876709
EPOCH 2, loss: 0.10067514647894046, acc: 0.9641509652137756
EPOCH 3, loss: 0.09926217767026495, acc: 0.9622641801834106
EPOCH 4, loss: 0.10824348635095007, acc: 0.9566037654876709
EPOCH 5, loss: 0.07762865974184345, acc: 0.9698113203048706


Best Acc 는0.96%로 상당히 높은 정확도를 보여주고 있습니다. 훈련데이터로는 학습이 잘되었다고 할 수 있습니다. <br />
이제 테스트 용도의데이터를 이용하여 모델 정확도를 측정해 보아야 합니다.

In [19]:
import glob

In [20]:

def eval_model(model, dataloaders, device):
   
    acc_history = []
    best_acc = 0.0

    saved_models = glob.glob('./080289/chap06/data/catanddog/' + '*.pth')
    saved_models.sort()
    print('saved_model', saved_models)

    for model_path in saved_models:
        print('Loading model', model_path)

        model.load_state_dict(torch.load(model_path))
        model.eval()
        model.to(device)
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
              outputs = model(inputs)

            _, preds = torch.max(outputs.data, 1)           
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0
            running_corrects += preds.eq(labels.cpu()).int().sum()

        
           
            
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print('Acc: {:.4f}'.format(epoch_acc))
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
      

    
    return acc_history

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
val_acc_hist = eval_model(model_v2, test_data_loader , device)

saved_model ['./080289/chap06/data/catanddog\\00.pth', './080289/chap06/data/catanddog\\01.pth', './080289/chap06/data/catanddog\\02.pth', './080289/chap06/data/catanddog\\03.pth', './080289/chap06/data/catanddog\\04.pth']
Loading model ./080289/chap06/data/catanddog\00.pth


C:\Users\user\AppData\Local\Temp\ipykernel_3608\1991491234.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Acc: 1.0000
Loading model ./080289/chap06/data/catanddog\01.pth
Acc: 1.0000
Loading model ./080289/chap06/data/catanddog\02.pth
Acc: 1.0000
Loading model ./080289/chap06/data/catanddog\03.pth
Acc: 1.0000
Loading model ./080289/chap06/data/catanddog\04.pth
Acc: 1.0000
